In [6]:
from tifffile import imwrite
from tkinter import filedialog as fd
import os
from os.path import dirname, basename
from scipy.ndimage import gaussian_filter
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rich.progress import track

# matplotlib.use("Agg")
pd.options.mode.chained_assignment = None  # default='warn'

In [12]:
def spots2heatmap(df, scaling_factor, xedges, yedges, tracklength_threshold):
    df_single_frame_spots = df[df["trackID"].isna()]
    img_spots, _, _ = np.histogram2d(
        x=df_single_frame_spots["x"].to_numpy(float) * scaling_factor,
        y=df_single_frame_spots["y"].to_numpy(float) * scaling_factor,
        bins=(xedges, yedges),
    )

    lst_tracklength = []
    # tracks
    df_tracks = df[df["trackID"].notna()]
    all_trackID = df_tracks["trackID"].unique()
    lst_of_arr_x = []
    lst_of_arr_y = []
    for trackID in track(all_trackID, description="Reconstruction"):
        df_current = df_tracks[df_tracks["trackID"] == trackID]
        lst_tracklength.append(df_current.shape[0])
        # for short tracks, treat as spots
        if df_current.shape[0] <= tracklength_threshold:
            lst_of_arr_x.append(df_current["x"].to_numpy(float) * scaling_factor)
            lst_of_arr_y.append(df_current["y"].to_numpy(float) * scaling_factor)
            continue
        # for long tracks, randomly pick tracklength_threshold number of spots
        else:
            chosen_idx = np.random.choice(df_current.shape[0], tracklength_threshold)
            lst_of_arr_x.append(
                df_current.iloc[chosen_idx]["x"].to_numpy(float) * scaling_factor
            )
            lst_of_arr_y.append(
                df_current.iloc[chosen_idx]["y"].to_numpy(float) * scaling_factor
            )
            continue

    img_tracks, _, _ = np.histogram2d(
        x=np.hstack(lst_of_arr_x),
        y=np.hstack(lst_of_arr_y),
        bins=(xedges, yedges),
    )

    heatmap = img_spots + img_tracks

    return heatmap

In [23]:
folder = "/Users/GGM/Library/CloudStorage/Dropbox/UMich_PhD/Writing/PhD_Publications/Walter-Compartmentalized_Cytosol_under_Hyperosmotic/GEM_HOPS Cotracking/Representative image"
os.chdir(folder)
fname = "wholeFOV_GEM_spots_reformatted.csv"

time_cutoff = 0  # remove the first 200 frames of tracking
tracklength_threshold = 5  # distinguish long versus short tracks

um_per_pixel = 0.117
scaling_factor = 2
um_per_pixel_PAINT = um_per_pixel / scaling_factor
xpixels = 425
ypixels = 350
xedges = np.arange((xpixels + 1) * scaling_factor)
yedges = np.arange((ypixels + 1) * scaling_factor)

df = pd.read_csv(fname)
df = df[df["t"] >= time_cutoff]
heatmap = spots2heatmap(df, scaling_factor, xedges, yedges, tracklength_threshold)
imwrite(fname[:-4] + "-heatmap.tif", np.swapaxes(heatmap, 0, 1))

Output()

In [24]:
fname = "cropped-C2-GEM_spots_reformatted.csv"

time_cutoff = 0  # remove the first 200 frames of tracking
tracklength_threshold = 5  # distinguish long versus short tracks

um_per_pixel = 0.117
scaling_factor = 2
um_per_pixel_PAINT = um_per_pixel / scaling_factor
xpixels = 50
ypixels = 50
xedges = np.arange((xpixels + 1) * scaling_factor)
yedges = np.arange((ypixels + 1) * scaling_factor)

df = pd.read_csv(fname)
df = df[df["t"] >= time_cutoff]
heatmap = spots2heatmap(df, scaling_factor, xedges, yedges, tracklength_threshold)
imwrite(fname[:-4] + "-heatmap.tif", np.swapaxes(heatmap, 0, 1))

Output()